In [289]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt 
from matplotlib.collections import LineCollection

import seaborn as sns 
import plotly.express as px

from sklearn import datasets
from sklearn.cluster import AgglomerativeClustering
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans

from scipy.cluster.hierarchy import dendrogram, linkage
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler


# Lecture des fichiers

In [290]:
df_dispo_al = pd.read_csv("DisponibiliteAlimentaire_2017.csv")
df_pop = pd.read_csv("Population_2000_2018.csv")
df_stabPol = pd.read_csv('PoliticalStability_7-18-2023.csv')

# Traitement des données 

## df_dispo_al

In [291]:
df_dispo_al = df_dispo_al[df_dispo_al['Produit'] == 'Viande de Volailles']

In [292]:
df_dispo_al.Produit.unique()

array(['Viande de Volailles'], dtype=object)

In [293]:
df_dispo_al = df_dispo_al[["Zone","Élément","Valeur"]]

In [294]:
groups = []
for name, group in df_dispo_al.groupby("Élément"):
    group.drop(columns="Élément", inplace=True)
    group.rename(columns={"Valeur": name}, inplace=True)
    group.drop_duplicates("Zone", keep="first", inplace=True)
    group.set_index("Zone", inplace=True)
    groups.append(group)

df = pd.concat(groups, axis=1)
df.head()


,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
Zone,,,,,,,,,,,,,,,,,
Afrique du Sud,0.0,NaN,NaN,143.0,35.69,9.25,14.11,2118.0,63.0,514.0,2035.0,83.0,1667.0,0.0,NaN,NaN,-0.0
Algérie,0.0,NaN,NaN,22.0,6.38,1.50,1.97,277.0,0.0,2.0,264.0,13.0,275.0,0.0,NaN,NaN,0.0
Angola,0.0,NaN,NaN,35.0,10.56,2.22,3.60,319.0,0.0,277.0,315.0,2.0,42.0,0.0,NaN,2.0,-0.0
Antigua-et-Barbuda,2.0,NaN,0.0,233.0,54.10,17.55,17.77,7.0,0.0,7.0,5.0,NaN,0.0,0.0,NaN,NaN,0.0
Bahamas,0.0,NaN,9.0,182.0,43.17,13.33,14.61,26.0,NaN,24.0,16.0,NaN,6.0,0.0,NaN,NaN,4.0


In [295]:
df.shape

(172, 17)

In [296]:
df.columns

Index(['Alimentation pour touristes', 'Aliments pour animaux',
       'Autres utilisations (non alimentaire)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité intérieure', 'Exportations - Quantité',
       'Importations - Quantité', 'Nourriture', 'Pertes', 'Production',
       'Résidus', 'Semences', 'Traitement', 'Variation de stock'],
      dtype='object')

In [297]:
df = df[['Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité intérieure', 'Importations - Quantité', 'Nourriture',
       'Production', 'Variation de stock']]

In [298]:
df.drop(index = "France",inplace=True)

In [299]:
df.isna().sum()

Disponibilité alimentaire (Kcal/personne/jour)    0
Disponibilité intérieure                          2
Importations - Quantité                           2
Nourriture                                        2
Production                                        4
Variation de stock                                3
dtype: int64

Je remplace les valeur nan par la moyenne de la collonne 

In [300]:
df = df.fillna(df.mean())

In [301]:
df.isna().sum()

Disponibilité alimentaire (Kcal/personne/jour)    0
Disponibilité intérieure                          0
Importations - Quantité                           0
Nourriture                                        0
Production                                        0
Variation de stock                                0
dtype: int64

In [302]:
df.reset_index(inplace=True)

In [303]:
df.describe()

,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité intérieure,Importations - Quantité,Nourriture,Production,Variation de stock
count,171.00000,171.000000,171.000000,171.000000,171.000000,171.000000
mean,74.45614,682.355030,87.065089,652.147929,719.053892,12.660714
std,60.73435,2179.678539,183.342275,2129.300298,2478.033531,73.779228
min,0.00000,2.000000,0.000000,2.000000,0.000000,-119.000000
25%,22.00000,31.000000,3.000000,29.000000,14.500000,0.000000
50%,63.00000,102.000000,16.000000,101.000000,72.000000,0.000000
75%,106.00000,373.500000,83.000000,376.000000,478.000000,8.000000
max,243.00000,18266.000000,1069.000000,18100.000000,21914.000000,859.000000


In [304]:
print("df ",df.Zone.unique(), "pop",df_pop.Zone.unique(),"stabPol",df_stabPol.Zone.unique())

df  ['Afrique du Sud' 'Algérie' 'Angola' 'Antigua-et-Barbuda' 'Bahamas'
 'Bangladesh' 'Barbade' 'Belize' 'Bolivie (État plurinational de)'
 'Botswana' 'Burkina Faso' 'Cabo Verde' 'Cambodge' 'Cameroun'
 'Chine - RAS de Hong-Kong' 'Chine - RAS de Macao' 'Chine, continentale'
 'Chypre' 'Colombie' 'Congo' 'Costa Rica' 'Cuba' 'Dominique' 'Égypte'
 'Équateur' 'Éthiopie' 'Fidji' 'Grenade' 'Guatemala' 'Guinée' 'Haïti'
 'Honduras' 'Îles Salomon' 'Inde' 'Indonésie' 'Iraq' 'Islande' 'Jamaïque'
 'Kenya' 'Kiribati' 'Madagascar' 'Malawi' 'Maldives' 'Maurice' 'Mexique'
 'Mozambique' 'Myanmar' 'Népal' 'Niger' 'Nigéria' 'Nouvelle-Calédonie'
 'Ouganda' 'Pakistan' 'Pérou' 'Philippines' 'Polynésie française'
 'République centrafricaine' 'République dominicaine'
 'République-Unie de Tanzanie' 'Rwanda' 'Sainte-Lucie'
 'Saint-Kitts-et-Nevis' 'Saint-Vincent-et-les Grenadines' 'Samoa'
 'Sao Tomé-et-Principe' 'Sénégal' 'Sierra Leone' 'Soudan' 'Sri Lanka'
 'Tadjikistan' 'Tchad' 'Thaïlande' 'Trinité-et-Tobago' 'V

## //

## df_pop

df_dispo_al représente les donnée de l'année 2017 je filtre donc df_pop sur 2017

In [305]:
df_pop = df_pop[df_pop.Année == 2017]

In [306]:
df_pop.Année.unique()

array([2017], dtype=int64)

In [307]:
df_pop.columns

Index(['Code Domaine', 'Domaine', 'Code zone', 'Zone', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'Note'],
      dtype='object')

In [308]:
df_pop = df_pop[["Zone","Valeur"]]

In [309]:
df_pop.Zone.duplicated().sum()

0

In [310]:
df_pop[df_pop["Zone"] == "France"]

,Zone,Valeur
1325,France,64842.509


In [311]:
df_pop.drop(index = 1325,inplace=True)

In [312]:
df_pop.rename(columns={"Valeur" : "Population"},inplace=True)

In [313]:
df_pop['Population'] = df_pop["Population"]*1000

In [314]:
df_pop.head()

,Zone,Population
17,Afghanistan,36296113.0
36,Afrique du Sud,57009756.0
55,Albanie,2884169.0
74,Algérie,41389189.0
93,Allemagne,82658409.0


In [315]:
df_pop.Population.sum()

7483291602.0

In [316]:
df_pop.describe()

,Population
count,2.350000e+02
mean,3.184379e+07
std,1.321589e+08
min,7.930000e+02
25%,3.788515e+05
50%,5.110695e+06
75%,1.885283e+07
max,1.421022e+09


## //

## df_stabPol

In [317]:
df_stabPol = df_stabPol[df_stabPol['Année'] == 2017]

In [318]:
df_stabPol.columns

Index(['Code Domaine', 'Domaine', 'Code zone', 'Zone', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'Note'],
      dtype='object')

In [319]:
df_stabPol = df_stabPol[["Zone","Valeur"]]

In [400]:
df_stabPol[df_stabPol['Zone'] == "France"]

,Zone,stabilite pol
1360,France,0.28


In [402]:
df_stabPol.drop(index = 1360 , inplace = True)

In [404]:
df_stabPol.head()

,Zone,stabilite pol
16,Pays-Bas (Royaume des),0.92
37,Afghanistan,-2.80
58,Afrique du Sud,-0.28
79,Albanie,0.38
100,Algérie,-0.92


In [405]:
df_stabPol.rename(columns={'Valeur': 'stabilite pol'},inplace=True)

In [406]:
df_stabPol.describe()

,stabilite pol
count,195.000000
mean,-0.068256
std,0.997414
min,-2.940000
25%,-0.660000
50%,0.030000
75%,0.760000
max,1.920000


In [407]:
df_pop.shape

(235, 2)

In [408]:
df.shape

(171, 7)

# Merge

In [409]:
df_inner = pd.merge(df,df_pop, how='inner',on='Zone')

In [410]:
df_inner.shape

(171, 8)

In [411]:
df_inner = pd.merge(df_inner,df_stabPol, how='left',on='Zone')

In [412]:
df_inner.shape

(171, 9)

In [413]:
df_stabPol.describe()

,stabilite pol
count,195.000000
mean,-0.068256
std,0.997414
min,-2.940000
25%,-0.660000
50%,0.030000
75%,0.760000
max,1.920000


In [414]:
df_inner[df_inner['stabilite pol'].isna()]

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité intérieure,Importations - Quantité,Nourriture,Production,Variation de stock,Population,stabilite pol
16,"Chine, continentale",59.0,18161.0,452.0,17518.0,18236.0,-50.0,1.421022e+09,NaN
50,Nouvelle-Calédonie,136.0,11.0,9.0,11.0,1.0,-1.0,2.771500e+05,NaN
55,Polynésie française,167.0,15.0,15.0,13.0,1.0,0.0,2.761020e+05,NaN
78,"Chine, Taiwan Province de",126.0,785.0,161.0,785.0,652.0,20.0,2.367455e+07,NaN
149,Pays-Bas,70.0,372.0,608.0,346.0,1100.0,-82.0,1.702135e+07,NaN


In [415]:
df_inner.describe()

,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité intérieure,Importations - Quantité,Nourriture,Production,Variation de stock,Population,stabilite pol
count,171.00000,171.000000,171.000000,171.000000,171.000000,171.000000,1.710000e+02,166.000000
mean,74.45614,682.355030,87.065089,652.147929,719.053892,12.660714,4.271310e+07,-0.077771
std,60.73435,2179.678539,183.342275,2129.300298,2478.033531,73.779228,1.535039e+08,0.902546
min,0.00000,2.000000,0.000000,2.000000,0.000000,-119.000000,5.204500e+04,-2.940000
25%,22.00000,31.000000,3.000000,29.000000,14.500000,0.000000,2.864792e+06,-0.637500
50%,63.00000,102.000000,16.000000,101.000000,72.000000,0.000000,9.729823e+06,-0.040000
75%,106.00000,373.500000,83.000000,376.000000,478.000000,8.000000,2.960962e+07,0.645000
max,243.00000,18266.000000,1069.000000,18100.000000,21914.000000,859.000000,1.421022e+09,1.600000


In [416]:
df_inner = df_inner.fillna(df_inner['stabilite pol'].mean())

In [417]:
df_inner.isna().sum()

Zone                                              0
Disponibilité alimentaire (Kcal/personne/jour)    0
Disponibilité intérieure                          0
Importations - Quantité                           0
Nourriture                                        0
Production                                        0
Variation de stock                                0
Population                                        0
stabilite pol                                     0
dtype: int64

In [418]:
df_inner.shape

(171, 9)

In [419]:
df_inner.isna().sum()

Zone                                              0
Disponibilité alimentaire (Kcal/personne/jour)    0
Disponibilité intérieure                          0
Importations - Quantité                           0
Nourriture                                        0
Production                                        0
Variation de stock                                0
Population                                        0
stabilite pol                                     0
dtype: int64

In [420]:
df_inner

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité intérieure,Importations - Quantité,Nourriture,Production,Variation de stock,Population,stabilite pol
0,Afrique du Sud,143.0,2118.0,514.0,2035.0,1667.0,-0.0,57009756.0,-0.28
1,Algérie,22.0,277.0,2.0,264.0,275.0,0.0,41389189.0,-0.92
2,Angola,35.0,319.0,277.0,315.0,42.0,-0.0,29816766.0,-0.38
3,Antigua-et-Barbuda,233.0,7.0,7.0,5.0,0.0,0.0,95426.0,0.75
4,Bahamas,182.0,26.0,24.0,16.0,6.0,4.0,381755.0,0.99
...,...,...,...,...,...,...,...,...,...
166,Turkménistan,15.0,27.0,9.0,26.0,20.0,2.0,5757667.0,-0.13
167,Turquie,71.0,1674.0,3.0,1674.0,2192.0,92.0,81116450.0,-1.79
168,Ukraine,78.0,1009.0,121.0,1006.0,1211.0,49.0,44487709.0,-1.87
169,Uruguay,33.0,33.0,3.0,31.0,33.0,0.0,3436641.0,1.05


In [421]:
# df_inner.set_index('Zone',inplace=True)

In [422]:
df_inner.head()

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité intérieure,Importations - Quantité,Nourriture,Production,Variation de stock,Population,stabilite pol
0,Afrique du Sud,143.0,2118.0,514.0,2035.0,1667.0,-0.0,57009756.0,-0.28
1,Algérie,22.0,277.0,2.0,264.0,275.0,0.0,41389189.0,-0.92
2,Angola,35.0,319.0,277.0,315.0,42.0,-0.0,29816766.0,-0.38
3,Antigua-et-Barbuda,233.0,7.0,7.0,5.0,0.0,0.0,95426.0,0.75
4,Bahamas,182.0,26.0,24.0,16.0,6.0,4.0,381755.0,0.99


In [423]:
df_inner.describe()

,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité intérieure,Importations - Quantité,Nourriture,Production,Variation de stock,Population,stabilite pol
count,171.00000,171.000000,171.000000,171.000000,171.000000,171.000000,1.710000e+02,171.000000
mean,74.45614,682.355030,87.065089,652.147929,719.053892,12.660714,4.271310e+07,-0.077771
std,60.73435,2179.678539,183.342275,2129.300298,2478.033531,73.779228,1.535039e+08,0.889174
min,0.00000,2.000000,0.000000,2.000000,0.000000,-119.000000,5.204500e+04,-2.940000
25%,22.00000,31.000000,3.000000,29.000000,14.500000,0.000000,2.864792e+06,-0.625000
50%,63.00000,102.000000,16.000000,101.000000,72.000000,0.000000,9.729823e+06,-0.050000
75%,106.00000,373.500000,83.000000,376.000000,478.000000,8.000000,2.960962e+07,0.625000
max,243.00000,18266.000000,1069.000000,18100.000000,21914.000000,859.000000,1.421022e+09,1.600000


In [424]:
df_inner.to_excel("traitementdonnee.xlsx")

#

In [442]:
df_inner

,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité intérieure,Importations - Quantité,Nourriture,Production,Variation de stock,Population,stabilite pol
0,Afrique du Sud,143.0,2118.0,514.0,2035.0,1667.0,-0.0,57009756.0,-0.28
1,Algérie,22.0,277.0,2.0,264.0,275.0,0.0,41389189.0,-0.92
2,Angola,35.0,319.0,277.0,315.0,42.0,-0.0,29816766.0,-0.38
3,Antigua-et-Barbuda,233.0,7.0,7.0,5.0,0.0,0.0,95426.0,0.75
4,Bahamas,182.0,26.0,24.0,16.0,6.0,4.0,381755.0,0.99
...,...,...,...,...,...,...,...,...,...
166,Turkménistan,15.0,27.0,9.0,26.0,20.0,2.0,5757667.0,-0.13
167,Turquie,71.0,1674.0,3.0,1674.0,2192.0,92.0,81116450.0,-1.79
168,Ukraine,78.0,1009.0,121.0,1006.0,1211.0,49.0,44487709.0,-1.87
169,Uruguay,33.0,33.0,3.0,31.0,33.0,0.0,3436641.0,1.05


#

#

#